In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
rootURL = "https://www.boredpanda.com/"
# Ads http:// to the given URL because it is the only way to check for server response
# If the user will add to the URL directions then they will be deleted
# Example: 'https://moz.com/learn/seo/external-link' will turn to 'https://moz.com/'
# https://stackoverflow.com/questions/32314304/check-if-an-url-is-relative-to-another-ie-they-are-on-the-same-host
if rootURL.split('//')[0] != 'http:' and rootURL.split('//')[0] != 'https:':
    rootURL = 'http://' + rootURL
rootURL = rootURL.split('/')[0] + '//' + rootURL.split('/')[2]    
source_code = requests.get(rootURL)
soup = BeautifulSoup(source_code.text, "lxml")
rootURL

'https://www.boredpanda.com'

In [3]:
import re
domainName = re.compile(r"https?://(www\.)?")
domainName = domainName.sub('', rootURL).strip().strip('/')
# domainName = domainName.split('.')[0]
domainName

'boredpanda.com'

In [4]:
alternativeURLs = []
alternativeURLs.append(domainName)
alternativeURLs.append("http://"+domainName)
alternativeURLs.append("http://"+domainName+"/")
alternativeURLs.append("http://"+domainName+"//")
alternativeURLs.append("https://"+domainName)
alternativeURLs.append("https://"+domainName+"/")
alternativeURLs.append("https://"+domainName+"//")
alternativeURLs.append("http://www."+domainName)
alternativeURLs.append("http://www."+domainName+"/")
alternativeURLs.append("http://www."+domainName+"//")
alternativeURLs.append("https://www."+domainName)
alternativeURLs.append("https://www."+domainName+"/")
alternativeURLs.append("https://www."+domainName+"//")
alternativeURLs

['boredpanda.com',
 'http://boredpanda.com',
 'http://boredpanda.com/',
 'http://boredpanda.com//',
 'https://boredpanda.com',
 'https://boredpanda.com/',
 'https://boredpanda.com//',
 'http://www.boredpanda.com',
 'http://www.boredpanda.com/',
 'http://www.boredpanda.com//',
 'https://www.boredpanda.com',
 'https://www.boredpanda.com/',
 'https://www.boredpanda.com//']

In [5]:
urlList = []
externalURLKeywords = ["http", "www", ".com"]
exceptions = ["#", " ", "tel:", "callto:", "mailto:", "wp-content", ".pdf", "twitter.com", "facebook.com", "google.com", "pinterest.com", ".jpg", ".jpeg", ".webm"]

In [6]:
def urlCrawler(url, current_page, max_pages):
    tempURL = ""
    print("now crawling page: "+url)
    current_page+=1
    if(current_page == 1):
        urlList.append(url)
    source_code = requests.get(url)
    if(source_code.status_code == 406):
        source_code = requests.get(rootURL, headers={"User-Agent": "XY"})
        status_code = source_code.status_code
        print("new status code: "+str(status_code))
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    a = soup.findAll("a")
    for eachURL in a:
        href = eachURL.get("href")
        if(href):
            if not any(x in href for x in exceptions):
                if (href not in alternativeURLs and rootURL+""+href not in alternativeURLs):
                    if(domainName in href):
                        print("appending "+href+" for domainName in href")
                        tempURL = href
                    elif(len(href)>1 and href[0] == "/"):
                        print("appending "+href+" for len(href)>1 and href[0] == "/"")
                        tempURL = rootURL+""+href
                    elif any(x in href for x in externalURLKeywords):
                        print("not appending "+href+" for any(x in href for x in externalURLKeywords)")
                        tempURL = ""
                    else:
                        print("appending "+href+" for else")
                        tempURL = rootURL+"/"+href
                if(tempURL!=""):
                    if (tempURL not in urlList and tempURL != rootURL):
                        urlList.append(tempURL)
                        print("appending: "+tempURL)
    if(current_page < max_pages and current_page < len(urlList)):
        urlCrawler(urlList[current_page], current_page, max_pages)

In [7]:
urlList = []
totalURL = 0
urlCrawler(rootURL, 0, 1000)
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

now crawling page: https://www.boredpanda.com
not appending https://itunes.apple.com/app/apple-store/id1001593357?pt=117833574&ct=Website%20Takeover&mt=8 for any(x in href for x in externalURLKeywords)
appending javascript:void(0); for else
appending: https://www.boredpanda.com/javascript:void(0);
appending javascript:void(0) for else
appending: https://www.boredpanda.com/javascript:void(0)
appending javascript:void(0) for else
appending javascript:void(0) for else
appending javascript:void(0) for else
appending javascript:void(0); for else
appending javascript:void(0); for else
appending javascript:void(0); for else


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [12]:
urlListDF = pd.DataFrame(urlList)
urlListDF.head(1000)

,0
0,https://www.visual-literacy.org
1,https://www.visual-literacy.org/about-us/
2,https://www.visual-literacy.org/articles/
3,https://www.visual-literacy.org/videos/
4,https://www.visual-literacy.org/books/
5,https://www.visual-literacy.org/maps/
6,https://www.visual-literacy.org/contact-us/
7,https://www.visual-literacy.org/pages/maps/map...
8,https://www.visual-literacy.org/pages/maps/boo...
9,https://www.visual-literacy.org/pages/maps/vis...


In [13]:
# urlName = rootURL
# urlName = urlName.replace("http://","")
# urlName = urlName.replace("https://","")
# urlName = urlName.replace("www.","")
# # urlName = urlName.replace("/","")
# urlListDF.to_csv(r'C:\Users\hiren\crawly\urlList_'+urlName+'.csv')

In [14]:
len(urlList)

14

In [15]:
urlList

['https://www.visual-literacy.org',
 'https://www.visual-literacy.org/about-us/',
 'https://www.visual-literacy.org/articles/',
 'https://www.visual-literacy.org/videos/',
 'https://www.visual-literacy.org/books/',
 'https://www.visual-literacy.org/maps/',
 'https://www.visual-literacy.org/contact-us/',
 'https://www.visual-literacy.org/pages/maps/mapping_tools_radar/radar.html',
 'https://www.visual-literacy.org/pages/maps/books_interactive_map.html',
 'https://www.visual-literacy.org/pages/maps/visualization_scholars/index.html',
 'https://www.visual-literacy.org/stairs_of_viz/stairs_of_viz.html',
 'https://www.visual-literacy.org/periodic_table/periodic_table.html',
 'https://www.visual-literacy.org/pages/maps/shneiderman_interface.html',
 'http://www.visual-literacy.org/periodic_table/periodic_table.html']

In [ ]:
linkslist = []
i=0
while (i<5):
    linklist = {}
    linklist['title'] = "title_"+str(i+1)
    linklist['url'] = "url_"+str(i+1)
    linkslist.append(linklist)
    i+=1

In [ ]:
linkslist

In [ ]:

linkslist[1]['url'] = 'updated_url_2'

In [ ]:
linkslist